In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
from joblib import Parallel, delayed
import quantstats as qs

In [3]:
closing = pd.read_csv(r'fechamento.csv', delimiter = ';')

# Set columns as ticker of the stock
closing.columns = closing.columns.str.replace('Fechamento\najust p/ prov\nEm moeda orig\n', '', regex = False)

# Remove spaces from tickers
closing.columns = closing.columns.str.strip()

# Set dates to datetime format
closing.Data = pd.to_datetime(closing.Data, dayfirst=True)

# Set dates as index
closing.set_index('Data', inplace=True)

# Replace "-" with NaNs
closing = closing.replace('-', np.nan)

# Make all observations numeric
closing = closing.apply(pd.to_numeric)

# Sort columns alphabetically
closing = closing.sort_index(axis=1)

# Drop columns with less than 92 non-NaN observations after 63 observations (roughly three months)
closing = closing.reindex(columns = closing[63:].dropna(thresh = 92, axis = 1).columns)

# Drop rows that only have NaN
closing.dropna(how = 'all', inplace=True)

In [4]:
ibov = pd.read_csv(r'ibov.csv', delimiter = ',')

ibov.columns = ibov.columns.str.replace('Fechamento\najust p/ prov\nEm moeda orig\n', '', regex = False)
ibov.columns = ibov.columns.str.strip()

ibov.Data = pd.to_datetime(ibov.Data, dayfirst=True)
ibov.set_index('Data', inplace=True)

ibov = ibov.replace('-', np.nan)
ibov = ibov.apply(pd.to_numeric)
ibov = ibov.sort_index(axis=1)

In [5]:
# Define returns from closing prices
retorno = closing.pct_change()

# Where closing prices is NaN, atribute NaN to return
retorno[closing.isnull() & retorno.eq(0)] = np.nan

# Calculate the standard deviation of return using the past six months, with a minimum of 21 valid observations
lowvol = retorno.rolling(window = 26, min_periods = 21).std().fillna(method="ffill", limit=3)

# Calculate the momentum of the closing prices using the return of the past six months
momentum = closing.pct_change(periods = 26)

# Where return is NaN, atribute NaN to momentum
momentum[retorno.isnull()] = np.nan

closing = closing[21:]

closing = closing.iloc[::5]
lowvol = lowvol.reindex(columns = closing.columns).loc[closing.index]
momentum = momentum.reindex(columns = closing.columns).loc[closing.index]

# Define returns from closing prices
retorno = closing.pct_change()

# Where closing prices is NaN, atribute NaN to return
retorno[closing.isnull() & retorno.eq(0)] = np.nan

# Select data that exists in closing
ibov = pd.merge(ibov, closing, left_index=True, right_index=True)['IBOV']
ibov = ibov.pct_change()

In [6]:
marketcap = pd.read_csv(r'marketcap.csv', delimiter = ',')
marketcap.columns = marketcap.columns.str.replace('Valor Mercado\nda empresa\nEm moeda orig\nem milhares\n', '', regex = False)
marketcap.columns = marketcap.columns.str.strip()

marketcap.Data = pd.to_datetime(marketcap.Data, dayfirst=True)
marketcap.set_index('Data', inplace=True)

marketcap = marketcap.replace('-', np.nan)
marketcap = marketcap.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', ''), errors = 'coerce'))

marketcap = marketcap.reindex(columns = closing.columns).loc[closing.index]


In [ ]:
evebit = pd.read_csv(r'evebit.csv', delimiter = ',')
evebit.columns = evebit.columns.str.replace('EV/EBIT emp\nEm moeda orig\nde 3 meses\nconsolid:sim*\n', '', regex = False)
evebit.columns = evebit.columns.str.strip()

evebit.Data = pd.to_datetime(evebit.Data, dayfirst=True)
evebit.set_index('Data', inplace=True)

evebit = evebit.replace('-', np.nan)
evebit = evebit.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', ''), errors = "coerce"))

evebit = evebit.reindex(columns=closing.columns).loc[closing.index]

In [8]:
roic = pd.read_csv(r'roic.csv', delimiter = ',')
roic.columns = roic.columns.str.replace('ROIC (IC fim) %\n de 3 meses\n consolid:sim*\n', '', regex = False)
roic.columns = roic.columns.str.strip()

roic.Data = pd.to_datetime(roic.Data, dayfirst=True)
roic.set_index('Data', inplace=True)

roic = roic.replace('-', np.nan)
roic = roic.replace(0, np.nan)
roic = roic.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', ''), errors = "coerce"))

roic = roic.resample('D').asfreq().fillna(method='ffill', limit=92)

roic = roic.reindex(columns=closing.columns).loc[closing.index].replace(0, np.nan)

In [9]:
lookback = 31

# Function for genarating regressions
def calculate_coeffs(stock):
    coeffs = []
    # 31 weeks is roughly six months
    for t in range(lookback, len(retorno.index)): 

        # Get the relevant data for the current stock up to the current period using data from the past defined by lookback
        stock_retorno = retorno.iloc[t-lookback-1:t][stock] 
        stock_momentum = momentum.iloc[t-lookback:t-1][stock]
        stock_lowvol = lowvol.iloc[t-lookback:t-1][stock]
        stock_marketcap = marketcap.iloc[t-lookback:t-1][stock]
        stock_evebit = evebit.iloc[t-lookback:t-1][stock]
        stock_roic = roic.iloc[t-lookback:t-1][stock]
        stock_ibov = ibov.iloc[t-lookback:t-1]

        # Combine the predictor variables into a single dataframe
        X = pd.concat([stock_momentum, stock_lowvol, stock_marketcap, stock_evebit, stock_roic, stock_ibov], axis=1, keys=["momentum", "lowvol", "marketcap", "evebit", "roic", "ibov"])
        
        # Add constant for the model

        # Remove remaining columns with NaN values
        valid_cols = ~np.isnan(X).any(axis=1)  & ~np.isnan(stock_retorno)
        X = X[valid_cols]
        stock_retorno = stock_retorno[valid_cols]

        # Skip this period if there are no valid cols
        if not valid_cols.any():

            # Set the coefficients of this period as NaNs
            coeffs.append(pd.Series([np.nan] * 6, index=["momentum", "lowvol", "marketcap", "evebit", "roic", "ibov"]))
            continue

        # Fit the multiple regression model
        model = sm.OLS(stock_retorno, X)
        results = model.fit()

        # Append the model coefficients to the list
        coeffs.append(results.params)
    return pd.Series(coeffs, index=retorno[lookback:(len(retorno.index))].index)

regression_coeffs = pd.DataFrame(index=retorno[lookback:(len(retorno.index))].index, columns=retorno.columns)

# n_jobs=-1 will use all available CPUs
coeff_results = Parallel(n_jobs=-1)(delayed(calculate_coeffs)(stock) for stock in retorno.columns)

for stock, coeffs in zip(retorno.columns, coeff_results):
    regression_coeffs[stock] = coeffs

In [10]:
# Calculate the predicted returns using the regression coefficients
indexes = regression_coeffs.index
predicted_returns_df = pd.DataFrame(index=indexes, columns=retorno.columns)

# Iterate over each stock
for stock in retorno.columns:
    # Get the relevant predictor variables for the current stock
    stock_momentum = momentum[stock].shift(1)
    stock_lowvol = lowvol[stock].shift(1)
    stock_marketcap = marketcap[stock].shift(1)
    stock_evebit = evebit[stock].shift(1)
    stock_roic = roic[stock].shift(1)
    stock_ibov = ibov.shift(1)

    # Combine the predictor variables into a single dataframe
    X = pd.concat([stock_momentum, stock_lowvol, stock_marketcap, stock_evebit, stock_roic, stock_ibov], axis = 1, 
                  keys=["momentum", "lowvol", "marketcap", "evebit", "roic", "ibov"])
    X = sm.add_constant(X)
    
    # Calculate the predicted returns using the regression coefficients
    predicted_returns_df[stock] = (X * pd.concat(regression_coeffs[stock].values, axis = 1, keys = indexes).T).sum(axis = 1)

In [11]:
# Predicted Sharpe Ratio
# Calculate the excess returns
excess_returns = predicted_returns_df

# Calculate the mean of excess returns
mean_excess_return = excess_returns.rolling(21).mean()

# Calculate the standard deviation of returns (volatility)
std_excess_return = excess_returns.rolling(21).std()

# Calculate the Sharpe Ratio
predicted_sharpe_ratio = mean_excess_return / std_excess_return

In [12]:
# Select the top 10 stocks based on predicted sharpe ratio for each week
top_stocks = predicted_sharpe_ratio.shift(1).apply(lambda row: row.nlargest(10).index, axis=1)

# Calculate the weights based on predicted sharpe ratio for the selected top 10 stocks
weights = pd.DataFrame(index=lowvol.index, columns=lowvol.columns)

# Iterate over each week
for date in top_stocks.index:
    # Get the selected top 10 stocks for the current week
    selected_stocks = top_stocks.loc[date]
    
    # Get the predicted sharpe for the selected top 10 stocks
    selected_sharpe = predicted_sharpe_ratio.loc[date, selected_stocks]
    
    # Normalize the predicted sharpe to get proportional weights (sum to 1)
    proportional_weights = selected_sharpe / selected_sharpe.sum()
    
    # Assign the proportional weights to the corresponding stocks for the current week
    weights.loc[date, selected_stocks] = proportional_weights


# Normalize the weights to ensure they sum up to 1 for each week
weights = weights.div(weights.sum(axis=1) + 1e-16, axis=0)

# Initialize a dataframe to store the portfolio returns
portfolio_returns = pd.DataFrame(index=predicted_sharpe_ratio.index, columns=['Portfolio_Return'])

# Iterate over each week
for date in top_stocks.index:
    # Get the actual returns and corresponding weights for the selected stocks on the current week
    selected_stocks = top_stocks.loc[date]
    actual_returns = retorno.loc[date, selected_stocks]
    selected_weights = weights.loc[date, selected_stocks]
    
    # Calculate the portfolio return as the weighted sum of the actual returns
    portfolio_return = (actual_returns * selected_weights).sum()
    
    # Store the portfolio return in the dataframe
    portfolio_returns.loc[date] = portfolio_return

# Convert the portfolio returns to numeric values
portfolio_returns = pd.to_numeric(portfolio_returns['Portfolio_Return'])[21:]

# Benchmark
ibov = pd.merge(ibov, portfolio_returns, left_index=True, right_index=True)['IBOV']

qs.reports.html(portfolio_returns, ibov)